<a href="https://colab.research.google.com/github/adity1234567/Deep-Dive-in-The-Field-of-Misinformation/blob/main/Deep_Learning_For_NLP_Zero_To_Transformers_%26_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert

Contents

In this Notebook I will start with the very Basics of RNN's and Build all the way to latest deep learning architectures to solve NLP problems. It will cover the Following:

- Simple RNN's

- Word Embeddings : Definition and How to get them
- LSTM's
- GRU's
- BI-Directional RNN's
- Encoder-Decoder Models (Seq2Seq Models)
- Attention Models
- Transformers - Attention is all you need
- BERT

In [6]:
import plotly.graph_objs as go  # for advanced customization of figures
import plotly.express as px     # for easier plotting
import plotly.figure_factory as ff  # for specialized charts


In [7]:
pip install tensorflow --upgrade


In [8]:
pip install numpy pandas scikit-learn tensorflow keras matplotlib seaborn plotly


In [9]:
import numpy as np


import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN, Dense, Activation, Dropout, Embedding, BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text

from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff


Configuring TPU's

For this version of Notebook we will be using TPU's as we have to built a BERT Model **bold text**

In [10]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [6]:
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'

Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv.zip
replace /content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv.zip
replace /content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv.zip


In [7]:
train = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv')


In [8]:
train.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [9]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [10]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            223549 non-null  object
 1   comment_text  223549 non-null  object
 2   toxic         223549 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.1+ MB


In [12]:

train=train.loc[:12000,:]
train.shape

(12001, 3)

We will check the maximum number of words that can be present in a comment , this will help us in padding later

This code helps find the longest comment (by word count) in the dataset.

In [13]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

Writing a function for getting auc score for validation

In [17]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc


In [18]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

# using keras tokenizer here

In [19]:
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=300,
                    input_shape=(max_len,)))  # Use input_shape instead of input_length
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 1500, 300)           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 100)                 │          40,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,089,301 (49.93 MB)

 Trainable params: 13,089,301 (49.93 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64 * strategy.num_replicas_in_sync)


Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.8747 - loss: 0.3512
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 373s 2s/step - accuracy: 0.9661 - loss: 0.1058
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.9959 - loss: 0.0146
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 1.0000 - loss: 0.0017
Epoch 5/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 1.0000 - loss: 5.4481e-04


In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip


--2025-03-23 06:30:15--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2025-03-23 06:30:15--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2025-03-23 06:30:15--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/

In [ ]:
!unzip '/content/glove.840B.300d.zip' -d '/content/drive/MyDrive/Health misinformation/glove'

Archive:  /content/glove.840B.300d.zip
  inflating: /content/drive/MyDrive/Health misinformation/glove/glove.840B.300d.txt  


In [14]:
from tqdm import tqdm  # Import tqdm for progress bar
import numpy as np
embeddings_index = {}
with open('/content/drive/MyDrive/Health misinformation/glove/glove.840B.300d.txt', 'r', encoding='utf-8') as f:
    for line in tqdm(f):  # Use tqdm to show progress bar
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray([float(val) for val in values[1:]])
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


2196017it [04:38, 7892.47it/s] 

Found 2196016 word vectors.


In [20]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 205044.01it/s]


In [29]:
print(len(word_index))
print(max_len)
print([embedding_matrix])

43496
1500
[array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.27204  , -0.06203  , -0.1884   , ...,  0.13015  , -0.18317  ,
         0.1323   ],
       [ 0.31924  ,  0.06316  , -0.27858  , ...,  0.082745 ,  0.097801 ,
         0.25045  ],
       ...,
       [ 0.080343 , -0.15662  ,  0.14681  , ..., -0.15832  , -0.1902   ,
         0.21084  ],
       [ 0.29739  , -0.63196  ,  0.7522   , ..., -0.030615 , -0.050674 ,
        -0.29829  ],
       [-0.14527  , -0.11402  ,  0.074765 , ...,  0.20067  , -0.0096736,
         0.39243  ]])]


In [31]:
model = Sequential(name="sequential_3")
model.add(Embedding(len(word_index) + 1, 300, weights=[embedding_matrix],
                    input_length=max_len, trainable=False))  # Ensure input shape is set

#model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.build(input_shape=(None, max_len))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3, ))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()  # Now it should show correct parameter counts


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ (None, 1500, 300)           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 100)                 │         160,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,209,601 (50.39 MB)

 Trainable params: 160,501 (626.96 KB)

 Non-trainable params: 13,049,100 (49.78 MB)

In [32]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64 * strategy.num_replicas_in_sync)


Epoch 1/5
146/150 ━━━━━━━━━━━━━━━━━━━━ 23s 6s/step - accuracy: 0.9022 - loss: 0.2882

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import roc_auc_score

scores = model.predict(xvalid_pad)
auc = roc_auc_score(yvalid, scores)  # Compute AUC score
print("AUC: %.2f%%" % (auc * 100))


76/76 ━━━━━━━━━━━━━━━━━━━━ 75s 968ms/step
AUC: 96.82%


In [ ]:
# Initialize scores_model list
scores_model = []

# Append the GRU model's AUC score
scores_model.append({'Model': 'GRU', 'AUC_Score': roc_auc_score(yvalid, scores)})

# Print the results
print(scores_model)


[{'Model': 'GRU', 'AUC_Score': np.float64(0.9681518466133601)}]


In [ ]:
scores_model

[{'Model': 'GRU', 'AUC_Score': np.float64(0.9681518466133601)}]

In [34]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential(name="sequential_4")
    model.add(Embedding(len(word_index) + 1,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True))  # Allow fine-tuning

    model.build(input_shape=(None, max_len))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ (None, 1500, 300)           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 600)                 │       1,442,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             601 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,492,101 (55.28 MB)

 Trainable params: 14,492,101 (55.28 MB)

 Non-trainable params: 0 (0.00 B)

CPU times: user 442 ms, sys: 426 ms, total: 868 ms
Wall time: 896 ms


still not undersatnding what is the difference!!!

In [35]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))
    model.build(input_shape=(None, max_len))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ (None, 1500, 300)           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 600)                 │       1,442,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             601 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,492,101 (55.28 MB)

 Trainable params: 1,443,001 (5.50 MB)

 Non-trainable params: 13,049,100 (49.78 MB)

CPU times: user 431 ms, sys: 217 ms, total: 648 ms
Wall time: 885 ms


In [2]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64 * strategy.num_replicas_in_sync)


NameError: name 'model' is not defined

# BERT

In [11]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers

from tokenizers import BertWordPieceTokenizer

In [12]:
train1 = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv')
valid = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv')
#sub = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/sample_submission.csv')

In [13]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)  # Changed pad_to to length
    all_ids = []

    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i + chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])

    return np.array(all_ids)

In [14]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE

strategy = tf.distribute.MirroredStrategy()
# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [15]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [16]:
x_train = fast_encode(train1.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train1.toxic.values
y_valid = valid.toxic.values

100%|██████████| 250/250 [00:24<00:00, 10.02it/s]


In [17]:

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [24]:
import tensorflow as tf
from transformers import TFDistilBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(transformer, max_len=512):
    """
    Function to build and compile the BERT model.
    """
    # Input layer for word IDs (input_ids)
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")

    # Input layer for attention mask (indicating which tokens should be attended to)
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # Pass inputs to the transformer model
    transformer_output = transformer(input_word_ids, attention_mask=attention_mask)[0]

    # Extract the [CLS] token's representation (first token)
    cls_token = transformer_output[:, 0, :]

    # Add a Dense layer for classification
    output = Dense(1, activation='sigmoid')(cls_token)

    # Create the model
    model = Model(inputs=[input_word_ids, attention_mask], outputs=output)

    # Compile the model with Adam optimizer, binary cross-entropy loss, and accuracy metric
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Load the pre-trained DistilBERT model
transformer_layer = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

# Set the maximum length for input sequence
MAX_LEN = 512

# Build the model
model = build_model(transformer_layer, max_len=MAX_LEN)

# Display model summary
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


ValueError: Exception encountered when calling layer 'tf_distil_bert_model_4' (type TFDistilBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_distil_bert_model_4' (type TFDistilBertModel):
  • input_ids=<KerasTensor shape=(None, 512), dtype=int32, sparse=False, name=input_word_ids>
  • attention_mask=<KerasTensor shape=(None, 512), dtype=int32, sparse=False, name=attention_mask>
  • head_mask=None
  • inputs_embeds=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [1]:
!pip install transformers --upgrade

In [21]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


ValueError: Exception encountered when calling layer 'tf_distil_bert_model_1' (type TFDistilBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_distil_bert_model_1' (type TFDistilBertModel):
  • input_ids=<KerasTensor shape=(None, 192), dtype=int32, sparse=False, name=input_word_ids>
  • attention_mask=None
  • head_mask=None
  • inputs_embeds=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False